# Caching

In [1]:
import os
from pyps import Basis, Hamiltonian
from pyps.stark import _radial_integral as radial_calc
from pyps.stark import _ang_integral as ang_calc
from pyps.zeeman import _zeeman as zeeman_calc
import matplotlib.pyplot as plt

In [2]:
# construct matrix
H = Hamiltonian(
    Basis.build(n_values=range(30, 40), MJ_values=[0, 1, 2, 3]), numerov=True
)

## In-memory cache


functools.lru_cache is used to avoid duplicated calculations when building matrices.

In [3]:
radial_calc.cache_info()

CacheInfo(hits=0, misses=0, maxsize=1000000, currsize=0)

In [4]:
ang_calc.cache_info()

CacheInfo(hits=0, misses=0, maxsize=1000000, currsize=0)

In [5]:
zeeman_calc.cache_info()

CacheInfo(hits=0, misses=0, maxsize=1000000, currsize=0)

In [6]:
# this takes a while...
H.stark(Fz=1.0)
H.zeeman(Bz=1.0)

Zeeman: 100%|███████████████████████████████| 5260/5260 [03:24<00:00, 25.73it/s]


<5260x5260 sparse matrix of type '<class 'numpy.float64'>'
	with 3575 stored elements in Compressed Sparse Row format>

In [7]:
radial_calc.cache_info()

CacheInfo(hits=118735, misses=3565, maxsize=1000000, currsize=3565)

In [8]:
ang_calc.cache_info()

CacheInfo(hits=119434, misses=2866, maxsize=1000000, currsize=2866)

In [9]:
zeeman_calc.cache_info()

CacheInfo(hits=3465, misses=445, maxsize=1000000, currsize=445)

After the first computation, the sparse matrices of an instance of `Hamiltonian` are stored as attributes, `H._e0_matrix`, `H._stark_z_matrix` and `H._zeeman_matrix`.   These will be reused by the instance for subsequent calculations and can be saved to disc for long term storage (see below).  

To delete the matrices, call the method `Hamiltonian.reset()`.

## Long-term cache

`joblib.dumps()` can be used to store an instance of Hamiltonian (including cached matrices).

In [10]:
from joblib import dump, load

In [11]:
cache_file = "Rydberg.joblib"
dump(H, cache_file)

['Rydberg.joblib']

In [12]:
H = load(cache_file)

In [13]:
H.stark(Fz=1.0)

<5260x5260 sparse matrix of type '<class 'numpy.float64'>'
	with 79880 stored elements in Compressed Sparse Row format>

In [14]:
H.zeeman(Bz=1.0)

<5260x5260 sparse matrix of type '<class 'numpy.float64'>'
	with 3575 stored elements in Compressed Sparse Row format>